In [10]:
!pip install geopy

In [21]:
import requests
import pandas as pd

# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Query to fetch schools, hospitals, and parks in King County
overpass_query = """
[out:json];
area["name"="United States"]["admin_level"="2"]->.country;
area["name"="Washington"]["admin_level"="4"]->.state;
area["name"="King County"]["admin_level"="6"]->.boundary;
(
 // Hospital
  node["amenity"="hospital"](area.boundary);
  way["amenity"="hospital"](area.boundary);
  relation["amenity"="hospital"](area.boundary);
  
  // Universities or Colleges
  node["amenity"="university"](area.boundary);
  way["amenity"="university"](area.boundary);
  relation["amenity"="university"](area.boundary);
  node["amenity"="college"](area.boundary);
  way["amenity"="college"](area.boundary);
  relation["amenity"="college"](area.boundary);
  
  // Transit Centers
  node["public_transport"="station"](area.boundary);
  way["public_transport"="station"](area.boundary);
  relation["public_transport"="station"](area.boundary);

);
out center;
"""

# Initialize an empty list for amenities
amenities = []

try:
    # Fetch data from Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()  # Raise an exception for HTTP errors

    # Parse the JSON response
    amenities_data = response.json()

    # Extract relevant details
    amenities = [
        {
            'name': element['tags'].get('name', 'Unknown'),
            'type': element['tags'].get('amenity', element['tags'].get('leisure', 'Unknown')),
            'latitude': element['lat'] if element['type'] == 'node' else element['center']['lat'],
            'longitude': element['lon'] if element['type'] == 'node' else element['center']['lon']
        }
        for element in amenities_data.get('elements', [])
        if 'tags' in element
    ]

    # Create a DataFrame from the amenities list
    amenities_df = pd.DataFrame(amenities)

except requests.exceptions.RequestException as e:
    print(f"Network error: {e}")
    amenities_df = pd.DataFrame(amenities)
except ValueError as e:
    print(f"Error parsing JSON: {e}")
    amenities_df = pd.DataFrame(amenities)

print(amenities_df)

# Save the results to a CSV file
amenities_df.to_csv("king_county_amenities.csv", index=False)
print("Amenities data saved to 'king_county_amenities.csv'")


                                        name            type   latitude  \
0                                  Tahlequah  ferry_terminal  47.332006   
1                              Vashon Island  ferry_terminal  47.510915   
2                                 Fauntleroy  ferry_terminal  47.523180   
3                                    Unknown         Unknown  47.444876   
4                  Navos West Seattle Campus        hospital  47.534445   
..                                       ...             ...        ...   
166                       Seattle University      university  47.609510   
167                        Bastyr University         college  47.730662   
168  Lake Washington Institute of Technology         college  47.705005   
169              Seattle Children's Hospital        hospital  47.663096   
170              Cornish College of the Arts         college  47.617484   

      longitude  
0   -122.507785  
1   -122.463838  
2   -122.396493  
3   -121.426365  
4   -122.